# Лекция 5: 

# Практика — Оконные функции и векторизация текста

---

In [22]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.sql.types import *
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF
from pyspark.ml import Pipeline
from pyspark.sql.functions import col, length, split, size, substring, countDistinct, min, max, avg
from pyspark.sql.functions import sqrt
from pyspark.sql.functions import when
import time
import os

spark = SparkSession.builder \
    .appName("Stable_Spark") \
    .master("local[*]") \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memory", "2g") \
    .config("spark.memory.fraction", "0.8") \
    .config("spark.memory.storageFraction", "0.3") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .config("spark.driver.maxResultSize", "2g") \
    .getOrCreate()

spark.sparkContext.setLogLevel("WARN")

###  Загрузка данных из Parquet

In [23]:
PARQUET_PATH = "data/processed/reviews.parquet"

df = spark.read.parquet(PARQUET_PATH)
print(f"Загружено из: {PARQUET_PATH}")
print(f"Количество записей: {df.count():,}")
    
required_cols = ['review_text', 'sentiment', 'film_id', 'review_num']
missing_cols = [col for col in required_cols if col not in df.columns]

if missing_cols:
    print(f"Внимание: отсутствуют колонки {missing_cols}")
    print("   Проверьте, что Лекция 4 выполнена полностью")
else:
    print(f"Все ключевые колонки присутствуют: {required_cols}")
    
if 'review_length' not in df.columns:
    df = df.withColumn('review_length', length(col('review_text')))
    print("Добавлена колонка: review_length")
    
if 'word_count' not in df.columns:
    df = df.withColumn('word_count', size(split(col('review_text'), '\\s+')))
    print("Добавлена колонка: word_count")
    
print("Схема данных для лекции 5:")
df.printSchema()

print("\nПример данных с признаками film_id и review_num:")
df.select("film_id", "review_num", "sentiment", 
              "review_length", substring("review_text", 1, 30).alias("preview")) \
      .orderBy("film_id", "review_num") \
      .show(5, truncate=False)

print("\nСтатистика по новым признакам:")
stats = df.agg(
        countDistinct("film_id").alias("unique_films"),
        min("review_num").alias("min_review_num"),
        max("review_num").alias("max_review_num"),
        avg("review_length").alias("avg_length")
        ).collect()[0]
    
print(f"  Уникальных фильмов: {stats['unique_films']:,}")
print(f"  Диапазон номеров рецензий: {stats['min_review_num']} - {stats['max_review_num']}")
print(f"  Средняя длина отзыва: {stats['avg_length']:.1f} символов")

Загружено из: data/processed/reviews.parquet


Количество записей: 109,998
Все ключевые колонки присутствуют: ['review_text', 'sentiment', 'film_id', 'review_num']
Схема данных для лекции 5:
root
 |-- review_text: string (nullable = true)
 |-- file_path: string (nullable = true)
 |-- sentiment: string (nullable = true)
 |-- film_id: integer (nullable = true)
 |-- review_num: integer (nullable = true)
 |-- review_length: integer (nullable = true)
 |-- word_count: integer (nullable = true)


Пример данных с признаками film_id и review_num:


+-------+----------+---------+-------------+------------------------------+
|film_id|review_num|sentiment|review_length|preview                       |
+-------+----------+---------+-------------+------------------------------+
|306    |1         |pos      |1722         |Фильмы начала 2000-годов всегд|
|306    |2         |pos      |999          |К студенту Джеймсу Клейтону об|
|306    |3         |pos      |1230         |Я симпатизирую Роджеру Дональд|
|306    |4         |neu      |3345         |С фильмом 'Рекрут' (The Recrui|
|306    |5         |pos      |926          |После первого просмотра фильма|
+-------+----------+---------+-------------+------------------------------+
only showing top 5 rows

Статистика по новым признакам:


  Уникальных фильмов: 9,065
  Диапазон номеров рецензий: 1 - 99
  Средняя длина отзыва: 2042.8 символов


### Часть 1. Оконные функции: анализ внутри групп без потери деталей

Ключевая концепция: 

В отличие от `GROUP BY`, который агрегирует данные и схлопывает строки, оконные функции позволяют выполнять вычисления над группой связанных строк (окном), сохраняя при этом каждую исходную строку нетронутой. Это идеально для анализа трендов, ранжирования и сравнений.

| Компонент окна | Описание | Пример на наших данных |
| :--- | :--- | :--- |
| Partition (`PARTITION BY`) | Разбивает данные на логические группы для независимых вычислений. | `PARTITION BY sentiment` — отдельные окна для `neg`, `pos`, `neu`. |
| Order (`ORDER BY`) | Определяет порядок строк внутри каждого раздела (партиции). | `ORDER BY review_length DESC` — сортировка от длинных к коротким отзывам. |
| Frame | Определяет, какие строки внутри партиции участвуют в расчете для текущей строки (например, "2 строки до текущей"). | `.rowsBetween(-2, 0)` — окно из трех строк: текущая и две предыдущие. |

Пример создания спецификации окна в PySpark:
```python
from pyspark.sql.window import Window
# Окно для ранжирования по длине отзыва внутри каждой тональности
window_spec = Window.partitionBy("sentiment").orderBy(col("review_length").desc())
```


### Проблема традиционных GROUP BY

```
ТРАДИЦИОННЫЙ GROUP BY:
┌─────────────┬────────────┐
│ sentiment   │ avg_length │
├─────────────┼────────────┤
│ neg         │ 245.6      │
│ pos         │ 312.8      │
│ neu         │ 198.4      │
└─────────────┴────────────┘

ЧТО ТЕРЯЕМ?
• Конкретные отзывы внутри групп
• Индивидуальные характеристики
• Возможность сравнения с групповыми метриками
```

Проблема: `GROUP BY` схлопывает данные, теряя детализацию. Мы получаем только агрегированные значения без возможности анализа отдельных записей.

###  Решение - оконные функции (Window Functions)

```
ОКОННЫЕ ФУНКЦИИ СОХРАНЯЮТ ДЕТАЛИЗАЦИЮ:
╔══════════╦════════════╦═══════════╦═══════════════════════════════════════╗
║ film_id  ║ review_num ║ sentiment ║ review_length ║ avg_in_group ║ rank ║
╠══════════╬════════════╬═══════════╬═══════════════╬═══════════════╬══════╣
║ 102      ║ 1          ║ pos       ║ 256           ║ 312.8         ║ 15   ║
║ 102      ║ 2          ║ pos       ║ 345           ║ 312.8         ║ 8    ║
║ 102      ║ 3          ║ pos       ║ 289           ║ 312.8         ║ 12   ║
║ 203      ║ 1          ║ neg       ║ 198           ║ 245.6         ║ 22   ║
╚══════════╩════════════╩═══════════╩═══════════════╩═══════════════╩══════╝

ПРЕИМУЩЕСТВА:
✓ Сохраняются все исходные строки
✓ Добавляются групповые метрики
✓ Возможность ранжирования внутри групп
✓ Сравнение с групповыми средними
```

Ключевая концепция: Оконные функции выполняют вычисления над группой связанных строк (окном), сохраняя каждую исходную строку нетронутой.


### Архитектура оконной функции - три компонента

```
ОКНО = PARTITION  +  ORDER   +   FRAME
          ↓            ↓         ↓
      Разбиваем   Сортируем   Определяем границы
```

#### Компонент 1: PARTITION BY (разбиение)
```
ЧТО ДЕЛАЕТ:
Разбивает данные на логические группы для независимых вычислений

ПРАКТИЧЕСКИЙ ПРИМЕР НА НАШИХ ДАННЫХ:
• PARTITION BY sentiment → отдельные окна для neg, pos, neu
• PARTITION BY film_id → анализ внутри каждого фильма
• PARTITION BY film_id, sentiment → комбинированное разбиение

ВИЗУАЛИЗАЦИЯ:
Данные → [neg отзывы] [pos отзывы] [neu отзывы]
             ↓             ↓            ↓
         Окно 1        Окно 2       Окно 3
```

#### Компонент 2: ORDER BY (упорядочивание)
```
ЧТО ДЕЛАЕТ:
Определяет порядок строк внутри каждого раздела (партиции)

ПРАКТИЧЕСКИЙ ПРИМЕР НА НАШИХ ДАННЫХ:
• ORDER BY review_length DESC → от длинных к коротким
• ORDER BY review_num ASC → по порядку рецензий
• ORDER BY review_date DESC → от новых к старым

ЗАЧЕМ НУЖНО:
1. Для ранжирующих функций (row_number, rank)
2. Для функций сдвига (lag, lead)
3. Для определения порядка в скользящих окнах
```

#### Компонент 3: FRAME (границы окна)
```
ЧТО ДЕЛАЕТ:
Определяет, какие строки внутри партиции участвуют в расчете
для текущей строки

ТИПЫ ФРЕЙМОВ:
1. ROWS BETWEEN - физические строки
   • .rowsBetween(-2, 0) → текущая + 2 предыдущие
   • .rowsBetween(Window.unboundedPreceding, 0) → все предыдущие

2. RANGE BETWEEN - логические значения
   • .rangeBetween(-100, 100) → значения в диапазоне ±100

ПРАКТИЧЕСКИЙ ПРИМЕР:
• Для скользящего среднего: .rowsBetween(-2, 0)
• Для накопительной суммы: .rowsBetween(Window.unboundedPreceding, 0)
```

###  Сравнение оконных функций с другими подходами

```
СРАВНИТЕЛЬНАЯ ТАБЛИЦА:

| Подход              | Сохраняет строки | Групповые метрики | Сравнение строк | Производительность |
|---------------------|------------------|-------------------|-----------------|--------------------|
| GROUP BY            | Нет              | Да                | Нет             | Высокая            |
| JOIN с агрегатами   | Да               | Да                | Ограниченно     | Средняя            |
| Оконные функции     | Да               | Да                | Полностью       | Зависит от окна    |
| UDF (Python)        | Да               | Нет               | Нет             | Низкая             |

ПРЕИМУЩЕСТВА ОКОННЫХ ФУНКЦИЙ:
1. Эффективность: вычисления в рамках одного прохода по данным
2. Гибкость: различные типы окон и функций
3. Читаемость: декларативный SQL-подобный синтаксис
4. Оптимизация: Catalyst оптимизирует оконные вычисления
```

### Часть 2. Типы оконных функций и их применение для анализа отзывов

В практике вы будете использовать несколько ключевых типов функций.

1. Ранжирующие функции
Эти функции присваивают порядковый номер строке внутри ее партиции.
*   `row_number()`: Присваивает уникальный номер строке (1, 2, 3...). Даже если значения в столбце сортировки совпадают, номера будут разными. *Используется для выбора топ-N записей (например, 3 самых длинных отзыва в каждой категории).*
*   `rank()`: Присваивает одинаковый ранг одинаковым значениям, но оставляет "пропуски" в нумерации (1, 1, 3...).
*   `dense_rank()`: Как `rank()`, но без пропусков в нумерации (1, 1, 2...).

2. Аналитические функции (сдвига)
Позволяют сравнивать значения из разных строк.
*   `lag()`: Получает значение из предыдущей строки в рамках партиции. *Пример: сравнить длину текущего отзыва с предыдущим того же пользователя.*
*   `lead()`: Получает значение из следующей строки.

3. Агрегатные функции в окне
Позволяют рассчитать агрегаты (сумму, среднее) по окну, а не по всему датасету.
*   `sum()`, `avg()`, `min()`, `max()`: *Пример: `avg(review_length) OVER (PARTITION BY sentiment)` — добавит в каждую строку среднюю длину отзывов по ее тональности.*
*   Скользящее окно: Агрегация по динамическому окну (например, "последние 3 отзыва") задается фреймом `.rowsBetween(-2, 0)`.
*   Накопительный итог: Для расчета используйте фрейм `.rowsBetween(Window.unboundedPreceding, Window.currentRow)`.


##  Классификация оконных функций

```
ТРИ ОСНОВНЫХ КАТЕГОРИИ ОКОННЫХ ФУНКЦИЙ:

1. РАНЖИРУЮЩИЕ ФУНКЦИИ
   Назначение: упорядочивание строк внутри партиции
   Примеры: row_number(), rank(), dense_rank(), ntile()

2. АНАЛИТИЧЕСКИЕ ФУНКЦИИ (СДВИГА)
   Назначение: сравнение с соседними строками
   Примеры: lag(), lead(), first_value(), last_value()

3. АГРЕГАТНЫЕ ФУНКЦИИ В ОКНЕ
   Назначение: вычисление статистик по группе
   Примеры: sum(), avg(), min(), max(), count()

ОБЩИЙ ПРИНЦИП:
Все эти функции работают НАД окном, но не изменяют исходные данные
```

## Ранжирующие функции - визуальное сравнение

### row_number() vs rank() vs dense_rank()

```
ИСХОДНЫЕ ДАННЫЕ (отсортированы по review_length DESC):
film_id | review_num | review_length | sentiment
--------|------------|---------------|-----------
102     | 5          | 450           | pos
102     | 3          | 450           | pos    ← одинаковые значения
102     | 1          | 380           | neg
102     | 2          | 380           | neg    ← одинаковые значения
102     | 4          | 290           | neu

РЕЗУЛЬТАТЫ РАНЖИРОВАНИЯ:
row_number()        rank()           dense_rank()
----------------    --------------   ----------------
1 (450) ←          1 (450) ←         1 (450) ←
2 (450)            1 (450)           1 (450)
3 (380)            3 (380) ←         2 (380) ←
4 (380)            3 (380)           2 (380)
5 (290)            5 (290)           3 (290)

КЛЮЧЕВЫЕ ОТЛИЧИЯ:
• row_number(): Всегда уникальные номера (1, 2, 3, 4, 5)
• rank():       Пропуски при одинаковых значениях (1, 1, 3, 3, 5)
• dense_rank(): Без пропусков (1, 1, 2, 2, 3)
```

##  Аналитические функции сдвига - lag() и lead()

```
КОНЦЕПЦИЯ "ОКНА ВРЕМЕНИ":
Текущая строка имеет доступ к соседним строкам в рамках партиции

    [предыдущая] ← lag(1)   [ТЕКУЩАЯ]   lead(1) → [следующая]
    [2 строки назад] ← lag(2)           lead(2) → [через 2 строки]

СИНТАКСИС:
lag(column, offset, default_value)  # offset - на сколько строк назад
lead(column, offset, default_value) # offset - на сколько строк вперед

ПРИМЕР НА НАШИХ ДАННЫХ:
Окно: Window.partitionBy("film_id").orderBy("review_num")

Исходные данные:
review_num | review_length | sentiment
-----------|---------------|----------
1          | 256           | pos
2          | 345           | neg      ← текущая строка
3          | 289           | pos

Результат для строки 2:
lag("review_length", 1)    = 256    (значение из строки 1)
lag("sentiment", 1)        = "pos"  (sentiment из строки 1)
lead("review_length", 1)   = 289    (значение из строки 3)
lead("sentiment", 1)       = "pos"  (sentiment из строки 3)
```

## Агрегатные функции в окне - базовые агрегаты

```
ОТЛИЧИЕ ОТ ГРУППИРОВКИ:
GROUP BY:                 ОКОННЫЕ ФУНКЦИИ:
[схлопывает строки]      [сохраняет все строки]
┌─────┬──────┐           ╔══════╦══════╦═══════════╗
│ cat │ avg  │           ║ id   ║ cat  ║ avg_in_cat║
├─────┼──────┤           ╠══════╬══════╬═══════════╣
│ A   │ 25.6 │           ║ 1    ║ A    ║ 25.6      ║
│ B   │ 32.1 │           ║ 2    ║ A    ║ 25.6      ║
└─────┴──────┘           ║ 3    ║ B    ║ 32.1      ║
                         ║ 4    ║ B    ║ 32.1      ║
                         ╚══════╩══════╩═══════════╝

ПРИМЕР НА НАШИХ ДАННЫХ:
# Средняя длина отзывов по каждой тональности
window_sentiment = Window.partitionBy("sentiment")
df = df.withColumn("avg_length_by_sentiment", 
                  avg("review_length").over(window_sentiment))

# Количество отзывов по каждому фильму
window_film = Window.partitionBy("film_id")
df = df.withColumn("reviews_per_film", 
                  count("*").over(window_film))

РЕЗУЛЬТАТ КАЖДОЙ СТРОКИ:
film_id | sentiment | review_length | avg_length_by_sentiment | reviews_per_film
--------|-----------|---------------|-------------------------|-----------------
102     | pos       | 450           | 312.8 (среднее по pos)  | 15 (всего отзывов о фильме 102)
102     | pos       | 380           | 312.8 (среднее по pos)  | 15
203     | neg       | 290           | 245.6 (среднее по neg)  | 8
```

## Типы окон для агрегатов - фреймы

### Три основных типа фреймов:

1. ПОЛНАЯ ПАРТИЦИЯ (по умолчанию)  
   `.rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing)`  
   Все строки партиции участвуют в расчете  
   Пример: общее среднее по всей категории  

2. НАКОПИТЕЛЬНЫЙ ИТОГ (CUMULATIVE)  
   `.rowsBetween(Window.unboundedPreceding, Window.currentRow)`  
   Все ПРЕДЫДУЩИЕ строки + текущая  
   Пример: running total, cumulative average  

3. СКОЛЬЗЯЩЕЕ ОКНО (SLIDING/ROLLING)  
   `.rowsBetween(-2, 0)`    # 3 строки: текущая + 2 предыдущих  
   `.rowsBetween(-1, 1)`    # 3 строки: предыдущая + текущая + следующая  


## Накопительные агрегаты - cumulative window

КОНЦЕПЦИЯ: "Вычисления нарастающим итогом"

ПРИМЕР: Кумулятивная сумма длин отзывов по фильму

ОКНО:
```python
Window.partitionBy("film_id")
      .orderBy("review_num")
      .rowsBetween(Window.unboundedPreceding, Window.currentRow)
```

ДАННЫЕ:
review_num | review_length | cumulative_sum
-----------|---------------|---------------
1          | 256           | 256     ← 256
2          | 345           | 601     ← 256 + 345
3          | 289           | 890     ← 256 + 345 + 289
4          | 320           | 1210    ← 256 + 345 + 289 + 320

ПРАКТИЧЕСКОЕ ПРИМЕНЕНИЕ:
1. Накопительное количество: count(*).over(cumulative_window)
2. Бегущее среднее: avg(column).over(cumulative_window)
3. Доля от общей суммы: column / sum(column).over(full_window)

КОД ДЛЯ НАШИХ ДАННЫХ:

```python
# Накопительная статистика по фильмам
cumulative_window = Window.partitionBy("film_id") \
                         .orderBy("review_num") \
                         .rowsBetween(Window.unboundedPreceding, Window.currentRow)

df_cumulative = df.withColumn("running_total", sum("review_length").over(cumulative_window)) \
                  .withColumn("running_avg", avg("review_length").over(cumulative_window)) \
                  .withColumn("running_count", count("*").over(cumulative_window))
```

##  Скользящие агрегаты - rolling window

КОНЦЕПЦИЯ: "Окно фиксированного размера, скользящее по данным"

ПРИМЕР: Скользящее среднее по 3 последним отзывам

ОКНО:
```python
Window.partitionBy("film_id")
      .orderBy("review_num")
      .rowsBetween(-2, 0)  # Текущая + 2 предыдущих
```

ДАННЫЕ:
review_num | review_length | rolling_avg_3
-----------|---------------|--------------
1          | 256           | 256.0    ← (256)/1
2          | 345           | 300.5    ← (256 + 345)/2
3          | 289           | 296.7    ← (256 + 345 + 289)/3
4          | 320           | 318.0    ← (345 + 289 + 320)/3
5          | 310           | 306.3    ← (289 + 320 + 310)/3

ВИДЫ СКОЛЬЗЯЩИХ ОКОН:
1. Симметричное: .rowsBetween(-1, 1)  # предыдущая + текущая + следующая
2. Только назад: .rowsBetween(-3, 0)   # 4 последних (текущая + 3 предыдущих)
3. Только вперед: .rowsBetween(0, 2)   # 3 следующих (текущая + 2 следующих)

ПРАКТИЧЕСКОЕ ПРИМЕНЕНИЕ:  
• Сглаживание временных рядов  
• Обнаружение трендов  
• Фильтрация шума в данных  
• Анализ краткосрочных паттернов  


## Сводная таблица применения функций для анализа отзывов

```
| Функция        | Тип          | Практическое применение для отзывов              | Пример использования                           |
|----------------|--------------|--------------------------------------------------|------------------------------------------------|
| row_number()   | Ранжирующая  | Топ-3 самых длинных отзывов по фильму            | .filter(col("rank") <= 3)                      |
| rank()         | Ранжирующая  | Рейтинг фильмов по средней оценке с пропусками   | Анализ при равных средних значениях            |
| dense_rank()   | Ранжирующая  | Группировка фильмов по "уровню" популярности     | Категоризация (популярные/средние/непопулярные)|
| lag()          | Аналитическая| Сравнение отзыва с предыдущим того же автора     | Анализ эволюции мнения                         |
| lead()         | Аналитическая| Предсказание следующей тональности по паттернам  | Обнаружение трендов                            |
| avg()          | Агрегатная   | Средняя длина отзывов по жанру                   | Сравнение сложности отзывов между жанрами      |
| sum()          | Агрегатная   | Общий объем текста по фильму                     | Оценка "обсуждаемости" фильма                  |
| count()        | Агрегатная   | Количество отзывов за период                     | Анализ активности пользователей                |
| min()/max()    | Агрегатная   | Диапазон длин отзывов по категории               | Обнаружение аномалий/выбросов                  |

КОМБИНИРОВАННЫЕ СЦЕНАРИИ:
1. Сначала rank() для определения позиции
2. Затем lag() для сравнения с предыдущим
3. Наконец avg() в скользящем окне для сглаживания
```

### Создание оконных спецификаций

In [24]:
from pyspark.sql.window import Window
from pyspark.sql.functions import col, row_number, rank, dense_rank, lag, lead, \
                                  avg, sum, count, stddev, min, max, first_value, last_value

# 1. окно для анализа последовательности отзывов по фильмам
window_film_seq = Window.partitionBy("film_id").orderBy("review_num")

# 2. окно для ранжирования отзывов по длине внутри тональности
window_rank_sentiment = Window.partitionBy("sentiment").orderBy(col("review_length").desc())

# 3. окно для агрегации без сортировки
window_agg_sentiment = Window.partitionBy("sentiment")

# 4. скользящее окно для трендов
window_sliding = Window.partitionBy("film_id") \
                       .orderBy("review_num") \
                       .rowsBetween(-2, 0)  # текущая + 2 предыдущих

# 5. накопительное окно
window_cumulative = Window.partitionBy("film_id") \
                         .orderBy("review_num") \
                         .rowsBetween(Window.unboundedPreceding, Window.currentRow)

###  Ранжирующие функции

In [25]:
# применение: row_number(), rank(), dense_rank()
print("\n1. Ранжирование отзывов по длине внутри каждой тональности:")

df_ranked = df.withColumn("row_num", row_number().over(window_rank_sentiment)) \
              .withColumn("rank", rank().over(window_rank_sentiment)) \
              .withColumn("dense_rank", dense_rank().over(window_rank_sentiment))

# примеры для каждой тональности
for sentiment in ["pos", "neg", "neu"]:
    print(f"\n   Тональность '{sentiment}':")
    sample = df_ranked.filter(col("sentiment") == sentiment) \
                     .select("film_id", "review_num", "review_length", 
                             "row_num", "rank", "dense_rank") \
                     .orderBy("row_num") \
                     .limit(3)
    sample.show(truncate=False)
    


1. Ранжирование отзывов по длине внутри каждой тональности:

   Тональность 'pos':


+-------+----------+-------------+-------+----+----------+
|film_id|review_num|review_length|row_num|rank|dense_rank|
+-------+----------+-------------+-------+----+----------+
|778218 |10        |5000         |1      |1   |1         |
|195434 |96        |5000         |2      |1   |1         |
|102740 |10        |5000         |3      |1   |1         |
+-------+----------+-------------+-------+----+----------+


   Тональность 'neg':


+-------+----------+-------------+-------+----+----------+
|film_id|review_num|review_length|row_num|rank|dense_rank|
+-------+----------+-------------+-------+----+----------+
|1176159|8         |5000         |1      |1   |1         |
|22500  |2         |4996         |2      |2   |2         |
|423063 |40        |4995         |3      |3   |3         |
+-------+----------+-------------+-------+----+----------+


   Тональность 'neu':


+-------+----------+-------------+-------+----+----------+
|film_id|review_num|review_length|row_num|rank|dense_rank|
+-------+----------+-------------+-------+----+----------+
|260898 |80        |5000         |1      |1   |1         |
|521607 |24        |4998         |2      |2   |2         |
|661911 |35        |4997         |3      |3   |3         |
+-------+----------+-------------+-------+----+----------+



   Объяснение различий функций:

   • row_number() - всегда уникальные номера (1, 2, 3, 4, 5)  
   • rank() - одинаковые ранги при равных значениях, с пропусками (1, 1, 3, 3, 5)  
   • dense_rank() - одинаковые ранги без пропусков (1, 1, 2, 2, 3)  

###  Аналитические функции сдвига

   • Как часто меняется тональность?  
   • Есть ли тренд в длине отзывов?  

In [26]:
# lag() и lead() для анализа последовательности
print("\n2. Анализ последовательности отзывов в фильме:")

df_with_lag_lead = df.withColumn("prev_length", lag("review_length", 1).over(window_film_seq)) \
                     .withColumn("next_length", lead("review_length", 1).over(window_film_seq)) \
                     .withColumn("prev_sentiment", lag("sentiment", 1).over(window_film_seq)) \
                     .withColumn("next_sentiment", lead("sentiment", 1).over(window_film_seq)) \
                     .withColumn("length_change", col("review_length") - col("prev_length"))

print("   Использованные функции:")
print("   • lag(review_length, 1) - длина предыдущего отзыва")
print("   • lead(review_length, 1) - длина следующего отзыва")
print("   • lag(sentiment, 1) - тональность предыдущего отзыва")
print("   • lead(sentiment, 1) - тональность следующего отзыва")

print("\n   Пример для фильма ID=306:")
film_example = df_with_lag_lead.filter(col("film_id") == 306) \
                              .select("film_id", "review_num", "sentiment", 
                                      "review_length", "prev_length", "next_length",
                                      "prev_sentiment", "next_sentiment", "length_change") \
                              .orderBy("review_num") \
                              .limit(5)
film_example.show(truncate=False)



2. Анализ последовательности отзывов в фильме:
   Использованные функции:
   • lag(review_length, 1) - длина предыдущего отзыва
   • lead(review_length, 1) - длина следующего отзыва
   • lag(sentiment, 1) - тональность предыдущего отзыва
   • lead(sentiment, 1) - тональность следующего отзыва

   Пример для фильма ID=306:
+-------+----------+---------+-------------+-----------+-----------+--------------+--------------+-------------+
|film_id|review_num|sentiment|review_length|prev_length|next_length|prev_sentiment|next_sentiment|length_change|
+-------+----------+---------+-------------+-----------+-----------+--------------+--------------+-------------+
|306    |1         |pos      |1722         |NULL       |999        |NULL          |pos           |NULL         |
|306    |2         |pos      |999          |1722       |1230       |pos           |pos           |-723         |
|306    |3         |pos      |1230         |999        |3345       |pos           |neu           |231         

###  Агрегатные функции в окне

   Использованные функции:

   • avg() - средняя длина отзывов по тональности  
   • count() - количество отзывов по тональности  
   • stddev() - разброс длин отзывов  
   • min()/max() - минимальная/максимальная длина  

In [27]:
print("\n3. Базовые агрегатные функции (групповые статистики):")

df_with_aggregates = df.withColumn("avg_length_in_sentiment", 
                                   avg("review_length").over(window_agg_sentiment)) \
                       .withColumn("total_reviews_in_sentiment", 
                                   count("*").over(window_agg_sentiment)) \
                       .withColumn("std_length_in_sentiment", 
                                   stddev("review_length").over(window_agg_sentiment)) \
                       .withColumn("min_length_in_sentiment", 
                                   min("review_length").over(window_agg_sentiment)) \
                       .withColumn("max_length_in_sentiment", 
                                   max("review_length").over(window_agg_sentiment))

print("\n   Статистика по тональностям:")
stats_sample = df_with_aggregates.select("sentiment", "avg_length_in_sentiment",
                                        "total_reviews_in_sentiment", "std_length_in_sentiment") \
                                .distinct() \
                                .orderBy("avg_length_in_sentiment", ascending=False)
stats_sample.show(truncate=False)


3. Базовые агрегатные функции (групповые статистики):

   Статистика по тональностям:


+---------+-----------------------+--------------------------+-----------------------+
|sentiment|avg_length_in_sentiment|total_reviews_in_sentiment|std_length_in_sentiment|
+---------+-----------------------+--------------------------+-----------------------+
|neg      |2092.0192849014793     |16697                     |1045.0218599134985     |
|pos      |2040.0542425363421     |72987                     |1051.288871516476      |
|neu      |2012.4027764103573     |20314                     |1072.928410757921      |
+---------+-----------------------+--------------------------+-----------------------+



In [28]:
print("\n4. Скользящие агрегаты (скользящее окно):")

df_sliding_agg = df.withColumn("rolling_avg_3", 
                               avg("review_length").over(window_sliding)) \
                   .withColumn("rolling_sum_3", 
                               sum("review_length").over(window_sliding)) \
                   .withColumn("rolling_count_3", 
                               count("*").over(window_sliding))

print("   Скользящее среднее по 3 последним отзывам в фильме:")

print("\n   Пример для фильма ID=306:")
sliding_example = df_sliding_agg.filter(col("film_id") == 306) \
                               .select("film_id", "review_num", "review_length",
                                       "rolling_avg_3", "rolling_sum_3", "rolling_count_3") \
                               .orderBy("review_num") \
                               .limit(6)
sliding_example.show(truncate=False)


4. Скользящие агрегаты (скользящее окно):
   Скользящее среднее по 3 последним отзывам в фильме:

   Пример для фильма ID=306:


+-------+----------+-------------+------------------+-------------+---------------+
|film_id|review_num|review_length|rolling_avg_3     |rolling_sum_3|rolling_count_3|
+-------+----------+-------------+------------------+-------------+---------------+
|306    |1         |1722         |1722.0            |1722         |1              |
|306    |2         |999          |1360.5            |2721         |2              |
|306    |3         |1230         |1317.0            |3951         |3              |
|306    |4         |3345         |1858.0            |5574         |3              |
|306    |5         |926          |1833.6666666666667|5501         |3              |
|306    |6         |1970         |2080.3333333333335|6241         |3              |
+-------+----------+-------------+------------------+-------------+---------------+



In [29]:
print("\n5. Накопительные агрегаты (накопительное окно):")

df_cumulative = df.withColumn("cumulative_length", 
                              sum("review_length").over(window_cumulative)) \
                  .withColumn("cumulative_count", 
                              count("*").over(window_cumulative)) \
                  .withColumn("running_avg", 
                              avg("review_length").over(window_cumulative)) \
                  .withColumn("cumulative_positives",
                              sum(when(col("sentiment") == "pos", 1).otherwise(0))
                              .over(window_cumulative))

print("   Накопительная статистика по фильмам:")

# Пример для конкретного фильма
print("\n   Пример для фильма ID=306:")
cumulative_example = df_cumulative.filter(col("film_id") == 306) \
                                 .select("film_id", "review_num", "sentiment",
                                         "review_length", "cumulative_length",
                                         "cumulative_count", "running_avg",
                                         "cumulative_positives") \
                                 .orderBy("review_num") \
                                 .limit(5)
cumulative_example.show(truncate=False)



5. Накопительные агрегаты (накопительное окно):
   Накопительная статистика по фильмам:

   Пример для фильма ID=306:
+-------+----------+---------+-------------+-----------------+----------------+-----------+--------------------+
|film_id|review_num|sentiment|review_length|cumulative_length|cumulative_count|running_avg|cumulative_positives|
+-------+----------+---------+-------------+-----------------+----------------+-----------+--------------------+
|306    |1         |pos      |1722         |1722             |1               |1722.0     |1                   |
|306    |2         |pos      |999          |2721             |2               |1360.5     |2                   |
|306    |3         |pos      |1230         |3951             |3               |1317.0     |3                   |
|306    |4         |neu      |3345         |7296             |4               |1824.0     |3                   |
|306    |5         |pos      |926          |8222             |5               |1644.4     

Функции first_value() и last_value():  
   • first_value() - значение из первой строки окна  
   • last_value() - значение из последней строки окна  

In [30]:
print("\n6. Функции first_value() и last_value():")

df_first_last = df.withColumn("first_review_in_film", 
                              first("review_length").over(window_film_seq)) \
                  .withColumn("last_review_in_film", 
                              last("review_length").over(
                                  Window.partitionBy("film_id")
                                        .orderBy("review_num")
                                        .rowsBetween(Window.unboundedPreceding, 
                                                   Window.unboundedFollowing)
                              ))


print("\n   Пример сравнения с первым и последним отзывом:")
sample_comparison = df_first_last.filter(col("film_id") == 306) \
                                .select("film_id", "review_num", "review_length",
                                        "first_review_in_film", "last_review_in_film",
                                        (col("review_length") - col("first_review_in_film")).alias("diff_from_first"),
                                        (col("review_length") - col("last_review_in_film")).alias("diff_from_last")) \
                                .orderBy("review_num") \
                                .limit(4)
sample_comparison.show(truncate=False)


6. Функции first_value() и last_value():

   Пример сравнения с первым и последним отзывом:


+-------+----------+-------------+--------------------+-------------------+---------------+--------------+
|film_id|review_num|review_length|first_review_in_film|last_review_in_film|diff_from_first|diff_from_last|
+-------+----------+-------------+--------------------+-------------------+---------------+--------------+
|306    |1         |1722         |1722                |169                |0              |1553          |
|306    |2         |999          |1722                |169                |-723           |830           |
|306    |3         |1230         |1722                |169                |-492           |1061          |
|306    |4         |3345         |1722                |169                |1623           |3176          |
+-------+----------+-------------+--------------------+-------------------+---------------+--------------+



### Анализ "спорных" фильмов

Задача: Найти фильмы с наиболее равномерным распределением тональностей  
Определение: высокая 'спорность' = близкое количество neg/pos/neu отзывов  
Метрика: низкое стандартное отклонение в распределении тональностей  

Вычисляем метрики 'спорности':  
   Формулы расчета:  
   • total_reviews = neg + pos + neu  
   • avg_count = total_reviews / 3   
   • variance = Σ(count - avg_count)² / 3  
   • std_dev = √variance (чем меньше, тем равномернее)  
   • controversy_score = 1 / (std_dev + 1) (чем ближе к 1, тем спорнее)

In [31]:
from pyspark.sql.functions import sqrt, pow


print("\n1. Создаем сводку распределения тональностей по фильмам:")
sentiment_distribution = df.groupBy("film_id", "sentiment") \
    .agg(count("*").alias("count_by_sentiment")) \
    .groupBy("film_id") \
    .pivot("sentiment", ["neg", "pos", "neu"]) \
    .agg(first("count_by_sentiment")) \
    .fillna(0)  

print("   Распределение по тональностям для первых 5 фильмов:")
sentiment_distribution.orderBy("film_id").limit(5).show(truncate=False)

print("\n2. Вычисляем метрики 'спорности':")
controversial_films = sentiment_distribution \
    .withColumn("total_reviews", col("neg") + col("pos") + col("neu")) \
    .withColumn("avg_count", col("total_reviews") / 3) \
    .withColumn("variance", 
               (pow(col("neg") - col("avg_count"), 2) + 
                pow(col("pos") - col("avg_count"), 2) + 
                pow(col("neu") - col("avg_count"), 2)) / 3) \
    .withColumn("std_dev", sqrt(col("variance"))) \
    .withColumn("controversy_score", 1 / (col("std_dev") + 1))  # +1 чтобы избежать деления на 0

print("3. ТОП-10 САМЫХ 'СПОРНЫХ' ФИЛЬМОВ")
print("   (близкое распределение тональностей)")

top_controversial = controversial_films \
    .filter(col("total_reviews") >= 10) \
    .select("film_id", "neg", "pos", "neu", "total_reviews", 
           "std_dev", "controversy_score") \
    .withColumn("neg_percent", (col("neg") / col("total_reviews") * 100).cast("int")) \
    .withColumn("pos_percent", (col("pos") / col("total_reviews") * 100).cast("int")) \
    .withColumn("neu_percent", (col("neu") / col("total_reviews") * 100).cast("int")) \
    .orderBy(col("controversy_score").desc()) \
    .limit(10)

print("Фильмы с наиболее равномерным распределением тональностей:")
print("(минимум 10 отзывов)")
print("\n" + "-"*120)
print(f"{'Film ID':<8} {'Neg':<4} {'Pos':<4} {'Neu':<4} {'Total':<6} {'Neg%':<5} {'Pos%':<5} {'Neu%':<5} {'StdDev':<8} {'Score':<6}")
print("-"*120)

for row in top_controversial.collect():
    print(f"{row['film_id']:<8} {row['neg']:<4} {row['pos']:<4} {row['neu']:<4} "
          f"{row['total_reviews']:<6} {row['neg_percent']:<5} {row['pos_percent']:<5} "
          f"{row['neu_percent']:<5} {row['std_dev']:<8.2f} {row['controversy_score']:<6.3f}")

print("-"*120)

print("4. ТОП-10 САМЫХ 'ОДНОЗНАЧНЫХ' ФИЛЬМОВ")
print("   (преобладание одной тональности)")

top_unambiguous = controversial_films \
    .filter(col("total_reviews") >= 10) \
    .select("film_id", "neg", "pos", "neu", "total_reviews", 
           "std_dev", "controversy_score") \
    .withColumn("max_sentiment", 
               when(col("neg") >= col("pos"), 
                    when(col("neg") >= col("neu"), "neg").otherwise("neu"))
               .otherwise(when(col("pos") >= col("neu"), "pos").otherwise("neu"))) \
    .withColumn("max_count", 
               greatest(col("neg"), col("pos"), col("neu"))) \
    .withColumn("dominance_percent", (col("max_count") / col("total_reviews") * 100).cast("int")) \
    .orderBy("controversy_score") \
    .limit(10)

print("Фильмы с преобладанием одной тональности:")
print(f"{'Film ID':<8} {'Neg':<4} {'Pos':<4} {'Neu':<4} {'Total':<6} {'Dominant':<9} {'Dominance%':<11} {'StdDev':<8} {'Score':<6}")
print("-"*120)

for row in top_unambiguous.collect():
    print(f"{row['film_id']:<8} {row['neg']:<4} {row['pos']:<4} {row['neu']:<4} "
          f"{row['total_reviews']:<6} {row['max_sentiment']:<9} "
          f"{row['dominance_percent']:<11} {row['std_dev']:<8.2f} {row['controversy_score']:<6.3f}")

print("-"*120)

print("5. ОБЩАЯ СТАТИСТИКА ПО ВСЕМ ФИЛЬМАМ")

# Рассчитываем статистику по распределению спорности
controversy_stats = controversial_films \
    .filter(col("total_reviews") >= 5) \
    .agg(
        count("*").alias("total_films"),
        avg("controversy_score").alias("avg_controversy_score"),
        stddev("controversy_score").alias("std_controversy_score"),
        min("controversy_score").alias("min_controversy_score"),
        max("controversy_score").alias("max_controversy_score"),
        avg("total_reviews").alias("avg_reviews_per_film")
    ).collect()[0]

print(f"Всего фильмов (с ≥5 отзывами): {controversy_stats['total_films']:,}")
print(f"Средний показатель спорности: {controversy_stats['avg_controversy_score']:.3f}")
print(f"Разброс показателей спорности: {controversy_stats['std_controversy_score']:.3f}")
print(f"Минимальная спорность: {controversy_stats['min_controversy_score']:.3f}")
print(f"Максимальная спорность: {controversy_stats['max_controversy_score']:.3f}")
print(f"Среднее количество отзывов на фильм: {controversy_stats['avg_reviews_per_film']:.1f}")

print("\nРаспределение фильмов по уровню спорности:")
controversy_bins = [(0.0, 0.2), (0.2, 0.4), (0.4, 0.6), (0.6, 0.8), (0.8, 1.0)]

for low, high in controversy_bins:
    count = controversial_films \
        .filter((col("controversy_score") >= low) & (col("controversy_score") < high) & 
                (col("total_reviews") >= 5)) \
        .count()
    percentage = count / controversy_stats['total_films'] * 100
    bar_length = int(percentage / 2)  # масштабируем для отображения
    bar = "█" * bar_length
    
    label = f"{low:.1f}-{high:.1f}"
    if high == 1.0:
        label = f"{low:.1f}-{high:.0f}"
    
    print(f"  {label:7} | {bar} {count:4d} фильмов ({percentage:.1f}%)")

print("6. ПРИМЕРЫ ОТЗЫВОВ ДЛЯ САМЫХ СПОРНЫХ ФИЛЬМОВ")

# ID самых спорных фильмов
top_controversial_ids = [row["film_id"] for row in top_controversial.limit(3).collect()]

for film_id in top_controversial_ids:
    print(f"\nФильм ID={film_id}:")
    
    # получаем распределение по тональностям
    film_stats = controversial_films.filter(col("film_id") == film_id).first()
    
    print(f"  Распределение: Neg={film_stats['neg']}, Pos={film_stats['pos']}, Neu={film_stats['neu']}")
    print(f"  Всего отзывов: {film_stats['total_reviews']}")
    print(f"  Показатель спорности: {film_stats['controversy_score']:.3f}")
    
    print("  Примеры отзывов:")
    
    for sentiment in ["pos", "neg", "neu"]:
        sample_review = df.filter((col("film_id") == film_id) & (col("sentiment") == sentiment)) \
                         .select("review_num", "sentiment", 
                                 substring("review_text", 1, 80).alias("text_preview")) \
                         .orderBy("review_num") \
                         .limit(1) \
                         .first()
        
        if sample_review:
            print(f"    {sentiment.upper()}: Отзыв {sample_review['review_num']} - {sample_review['text_preview']}...")

print("Сохранение таблицы с метриками спорности фильмов...")

controversial_films.select(
    "film_id", "neg", "pos", "neu", "total_reviews",
    "avg_count", "variance", "std_dev", "controversy_score"
).write.mode("overwrite").parquet("data/processed/film_controversy_analysis.parquet")

print(f"✓ Сохранено: data/processed/film_controversy_analysis.parquet")
print(f"✓ Количество фильмов: {controversial_films.count():,}")

# сохраняем топ-100 самых спорных фильмов для быстрого доступа
top_100_controversial = controversial_films \
    .filter(col("total_reviews") >= 10) \
    .orderBy(col("controversy_score").desc()) \
    .limit(100)

top_100_controversial.write.mode("overwrite").parquet("data/processed/top_100_controversial_films.parquet")
print(f"✓ Сохранено: data/processed/top_100_controversial_films.parquet")
print(f"✓ Сохранено топ-100 самых спорных фильмов")


1. Создаем сводку распределения тональностей по фильмам:
   Распределение по тональностям для первых 5 фильмов:


+-------+---+---+---+
|film_id|neg|pos|neu|
+-------+---+---+---+
|306    |5  |31 |7  |
|325    |2  |78 |8  |
|331    |8  |60 |9  |
|335    |12 |77 |6  |
|342    |9  |74 |4  |
+-------+---+---+---+


2. Вычисляем метрики 'спорности':
3. ТОП-10 САМЫХ 'СПОРНЫХ' ФИЛЬМОВ
   (близкое распределение тональностей)
Фильмы с наиболее равномерным распределением тональностей:
(минимум 10 отзывов)

------------------------------------------------------------------------------------------------------------------------
Film ID  Neg  Pos  Neu  Total  Neg%  Pos%  Neu%  StdDev   Score 
------------------------------------------------------------------------------------------------------------------------


102221   5    5    5    15     33    33    33    0.00     1.000 
293837   4    4    4    12     33    33    33    0.00     1.000 
418113   5    5    5    15     33    33    33    0.00     1.000 
22635    4    4    4    12     33    33    33    0.00     1.000 
8065     6    6    6    18     33    33    33    0.00     1.000 
453615   7    6    7    20     35    30    35    0.47     0.680 
618381   3    3    4    10     30    30    40    0.47     0.680 
893627   4    4    3    11     36    36    27    0.47     0.680 
427272   4    5    5    14     28    35    35    0.47     0.680 
1128272  7    6    7    20     35    30    35    0.47     0.680 
------------------------------------------------------------------------------------------------------------------------
4. ТОП-10 САМЫХ 'ОДНОЗНАЧНЫХ' ФИЛЬМОВ
   (преобладание одной тональности)
Фильмы с преобладанием одной тональности:
Film ID  Neg  Pos  Neu  Total  Dominant  Dominance%  StdDev   Score 
--------------------------------------------

2950     1    98   0    99     pos       98          45.96    0.021 
42664    0    90   2    92     pos       97          41.96    0.023 
5330     1    90   3    94     pos       95          41.49    0.024 
414      1    88   3    92     pos       95          40.55    0.024 
77164    2    88   2    92     pos       95          40.54    0.024 
1991     2    88   5    95     pos       92          39.85    0.024 
89514    0    86   4    90     pos       95          39.63    0.025 
7097     2    88   6    96     pos       91          39.63    0.025 
259829   5    88   4    97     pos       90          39.36    0.025 
555      3    87   5    95     pos       91          39.14    0.025 
------------------------------------------------------------------------------------------------------------------------
5. ОБЩАЯ СТАТИСТИКА ПО ВСЕМ ФИЛЬМАМ


Всего фильмов (с ≥5 отзывами): 3,776
Средний показатель спорности: 0.238
Разброс показателей спорности: 0.160
Минимальная спорность: 0.021
Максимальная спорность: 1.000
Среднее количество отзывов на фильм: 26.5

Распределение фильмов по уровню спорности:


...

    NEG: Отзыв 1 - Вторая часть 'У холмов есть глаза' выходит в пик популярности Джейсона Вурхиза и...


    NEU: Отзыв 5 - Режиссёр Уэс Крейвен?! Да, действительно, он. Конечно, у каждого бывают не очень...
Сохранение таблицы с метриками спорности фильмов...


✓ Сохранено: data/processed/film_controversy_analysis.parquet


✓ Количество фильмов: 9,065


✓ Сохранено: data/processed/top_100_controversial_films.parquet
✓ Сохранено топ-100 самых спорных фильмов


### Часть 3. Векторизация текста: от слов к числам

Текст должен быть преобразован в числовые векторы, так как алгоритмы машинного обучения работают только с числами.

1. TF-IDF (Term Frequency-Inverse Document Frequency)
Это классический и эффективный метод, который оценивает важность слова в документе относительно всей коллекции документов.

| Компонент | Описание | Логика |
| :--- | :--- | :--- |
| Term Frequency (TF) | Как часто слово встречается в данном документе. | Частое слово в отзыве = потенциально важное для него. |
| Inverse Document Frequency (IDF) | Насколько редко слово встречается во всей коллекции документов. | Редкое слово в целом = более уникальное и информативное. |
| TF-IDF | Произведение TF и IDF. | Высокий вес получают слова, которые часты в конкретном документе, но редки в других. |

Реализация в PySpark: TF часто вычисляется с помощью `HashingTF`, который быстро сопоставляет слова с индексами фичей, а затем применяется `IDF`.

2. Word2Vec (семантические эмбеддинги)
Это более продвинутый подход, который создает "плотные" векторные представления слов, учитывая их смысловой контекст.
*   Принцип: Слова, встречающиеся в похожих контекстах (например, "отличный" и "прекрасный"), получают близкие по значению векторы.
*   Архитектуры:
    *   CBOW: Предсказывает слово по его контексту.
    *   Skip-gram: Предсказывает контекст по заданному слову (чаще используется).
*   Результат: На выходе получается модель, которая может находить семантически близкие слова (синонимы) с помощью `findSynonyms()`.


#### Базовая токенизация текста

In [34]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, RegexTokenizer
from pyspark.sql.functions import size, array_contains
from pyspark.sql.functions import col, size, avg, count as sql_count

# Способ 1: базовый Tokenizer (разделяет по пробелам)
tokenizer = Tokenizer(inputCol="review_text", outputCol="words_raw")
df_tokenized = tokenizer.transform(df)

print(f"   Токенизировано отзывов: {df_tokenized.count():,}")

print("\n   Примеры токенизации (первые 3 отзыва):")
samples = df_tokenized.select("film_id", "review_num", "sentiment", 
                              "review_text", "words_raw").limit(3).collect()

for i, sample in enumerate(samples):
    original_preview = sample["review_text"][:70] + "..." if len(sample["review_text"]) > 70 else sample["review_text"]
    words_preview = sample["words_raw"][:10]  # показываем первые 10 слов
    
    print(f"\n   Пример {i+1} (Фильм {sample['film_id']}, отзыв {sample['review_num']}):")
    print(f"   Тональность: {sample['sentiment']}")
    print(f"   Текст: {original_preview}")
    print(f"   Токены: {words_preview}")
    print(f"   Всего слов: {len(sample['words_raw'])}")

# статистика по токенам до очистки
print("\n   Статистика по токенам (до очистки):")
token_stats_raw = df_tokenized.withColumn("word_count_raw", size(col("words_raw"))) \
                             .groupBy("sentiment") \
                             .agg(
                                 avg("word_count_raw").alias("avg_words_raw"),
                                 stddev("word_count_raw").alias("std_words_raw"),
                                 min("word_count_raw").alias("min_words_raw"),
                                 max("word_count_raw").alias("max_words_raw")
                             ).orderBy("avg_words_raw", ascending=False)

token_stats_raw.show()

# анализ длины отзывов по фильмам
print("\n   Анализ длины отзывов по фильмам:")
film_word_stats = df_tokenized.groupBy("film_id") \
                             .agg(
                                 avg(size(col("words_raw"))).alias("avg_words_per_review"),
                                 sql_count("*").alias("total_reviews")
                             ).orderBy(col("avg_words_per_review").desc())

print("   Топ-5 фильмов с самыми длинными отзывами:")
film_word_stats.limit(5).show()

   Токенизировано отзывов: 109,998

   Примеры токенизации (первые 3 отзыва):

   Пример 1 (Фильм 841263, отзыв 86):
   Тональность: pos
   Текст: С момента появления первых фильмов от 'Марвел' прошло уже прилично вре...
   Токены: ['с', 'момента', 'появления', 'первых', 'фильмов', 'от', "'марвел'", 'прошло', 'уже', 'прилично']
   Всего слов: 776

   Пример 2 (Фильм 906042, отзыв 34):
   Тональность: pos
   Текст: Паддингтон - лучший медведь в кино? Этот вопрос меня уже долго мучает,...
   Токены: ['паддингтон', '-', 'лучший', 'медведь', 'в', 'кино?', 'этот', 'вопрос', 'меня', 'уже']
   Всего слов: 798

   Пример 3 (Фильм 230752, отзыв 2):
   Тональность: neg
   Текст: Абсурд.

Представьте себе Илью Муромца, вот он в первой серии едет в К...
   Токены: ['абсурд.', '', 'представьте', 'себе', 'илью', 'муромца,', 'вот', 'он', 'в', 'первой']
   Всего слов: 769

   Статистика по токенам (до очистки):


+---------+-----------------+------------------+-------------+-------------+
|sentiment|    avg_words_raw|     std_words_raw|min_words_raw|max_words_raw|
+---------+-----------------+------------------+-------------+-------------+
|      neg|323.0762412409415| 159.1177517092739|           11|          953|
|      pos| 313.285667310617|158.95934092206323|            8|         1130|
|      neu|307.6399035148174| 161.0270660152362|            9|          856|
+---------+-----------------+------------------+-------------+-------------+


   Анализ длины отзывов по фильмам:
   Топ-5 фильмов с самыми длинными отзывами:


+-------+--------------------+-------------+
|film_id|avg_words_per_review|total_reviews|
+-------+--------------------+-------------+
| 225362|               801.0|            1|
|1108925|               792.0|            1|
|  64538|               786.0|            1|
| 218483|               786.0|            1|
|   3653|               784.0|            1|
+-------+--------------------+-------------+



#### Расширенная токенизация с очисткой

In [35]:
from pyspark.sql.functions import lower, regexp_replace, trim

# предварительная очистка текста
print("   а) Предварительная очистка текста:")
df_cleaned = df_tokenized.withColumn(
    "cleaned_text",
    trim(lower(regexp_replace(col("review_text"), "[^а-яёa-z0-9\\s]", " ")))
)

# улучшенная токенизация с учетом русского языка
print("   б) Улучшенная токенизация для русского языка:")
regex_tokenizer = RegexTokenizer(
    inputCol="cleaned_text",
    outputCol="words_clean",
    pattern="\\s+",  # разделитель - один или более пробельных символов
    gaps=True
)

df_tokenized_clean = regex_tokenizer.transform(df_cleaned)

# удаляем пустые токены
df_tokenized_clean = df_tokenized_clean.withColumn(
    "words_filtered",
    array_remove(col("words_clean"), "")
)

# сравнение до и после очистки
print("\n   в) Сравнение до и после очистки:")
comparison_sample = df_tokenized_clean.select(
    "film_id", "review_num",
    substring("review_text", 1, 50).alias("original_preview"),
    substring("cleaned_text", 1, 50).alias("cleaned_preview"),
    col("words_raw").alias("tokens_raw"),
    col("words_filtered").alias("tokens_clean")
).limit(2).collect()

for i, sample in enumerate(comparison_sample):
    print(f"\n   Пример {i+1} (Фильм {sample['film_id']}):")
    print(f"   Оригинал: {sample['original_preview']}")
    print(f"   Очищенный: {sample['cleaned_preview']}")
    print(f"   Токены (сырые): {sample['tokens_raw'][:8]}")
    print(f"   Токены (очищенные): {sample['tokens_clean'][:8]}")
    
    diff = len(sample['tokens_raw']) - len(sample['tokens_clean'])
    if diff > 0:
        print(f"   Удалено символов/токенов: {diff}")

   а) Предварительная очистка текста:
   б) Улучшенная токенизация для русского языка:

   в) Сравнение до и после очистки:

   Пример 1 (Фильм 841263):
   Оригинал: С момента появления первых фильмов от 'Марвел' про
   Очищенный: момента появления первых фильмов от   арвел  прошл
   Токены (сырые): ['с', 'момента', 'появления', 'первых', 'фильмов', 'от', "'марвел'", 'прошло']
   Токены (очищенные): ['момента', 'появления', 'первых', 'фильмов', 'от', 'арвел', 'прошло', 'уже']
   Удалено символов/токенов: 32

   Пример 2 (Фильм 906042):
   Оригинал: Паддингтон - лучший медведь в кино? Этот вопрос ме
   Очищенный: аддингтон   лучший медведь в кино   тот вопрос мен
   Токены (сырые): ['паддингтон', '-', 'лучший', 'медведь', 'в', 'кино?', 'этот', 'вопрос']
   Токены (очищенные): ['аддингтон', 'лучший', 'медведь', 'в', 'кино', 'тот', 'вопрос', 'меня']
   Удалено символов/токенов: 31


#### Удаление стоп-слов

In [36]:
russian_stopwords = [
    "и", "в", "во", "не", "что", "он", "на", "я", "с", "со", "как", "а", "то", "все",
    "она", "так", "его", "но", "да", "ты", "к", "у", "же", "вы", "за", "бы", "по",
    "только", "ее", "мне", "было", "вот", "от", "меня", "еще", "нет", "о", "из", "ему",
    "теперь", "когда", "даже", "ну", "вдруг", "ли", "если", "уже", "или", "ни", "быть",
    "был", "него", "до", "вас", "нибудь", "опять", "уж", "вам", "ведь", "там", "потом",
    "себя", "ничего", "ей", "может", "они", "тут", "где", "есть", "надо", "ней", "для",
    "мы", "тебя", "их", "чем", "была", "сам", "чтоб", "без", "будто", "чего", "раз",
    "тоже", "себе", "под", "будет", "ж", "тогда", "кто", "этот", "того", "потому",
    "этого", "какой", "совсем", "ним", "здесь", "этом", "один", "почти", "мой", "тем",
    "чтобы", "нее", "сейчас", "были", "куда", "зачем", "всех", "никогда", "можно",
    "при", "наконец", "два", "об", "другой", "хоть", "после", "над", "больше", "тот",
    "через", "эти", "нас", "про", "всего", "них", "какая", "много", "разве", "три",
    "эту", "моя", "впрочем", "хорошо", "свою", "этой", "перед", "иногда", "лучше",
    "чуть", "том", "нельзя", "такой", "им", "более", "всегда", "конечно", "всю", "между"
]

film_stopwords = [
    "фильм", "кино", "смотреть", "режиссер", "актер", "актриса", "роль",
    "сюжет", "концовка", "кадр", "эпизод", "сцена", "персонаж", "герой",
    "картина", "лента", "просмотр", "кинолента"
]

# объединяем стоп-слова
custom_stopwords = russian_stopwords + film_stopwords
print(f"   Всего стоп-слов: {len(custom_stopwords)}")
print(f"   Из них общих: {len(russian_stopwords)}")
print(f"   Контекстных (фильмы): {len(film_stopwords)}")

print("\n   Примеры стоп-слов:")
print(f"   Общие: {russian_stopwords[:10]}")
print(f"   Контекстные: {film_stopwords}")

# инициализируем удаление стоп-слов
stopwords_remover = StopWordsRemover(
    inputCol="words_filtered",
    outputCol="words_without_stopwords",
    stopWords=custom_stopwords
)

# применяем удаление стоп-слов
df_no_stopwords = stopwords_remover.transform(df_tokenized_clean)

# анализ эффективности удаления стоп-слов
print("\n  Эффективность удаления стоп-слов:")

stopword_analysis = df_no_stopwords.withColumn(
    "words_before", size(col("words_filtered"))
).withColumn(
    "words_after", size(col("words_without_stopwords"))
).withColumn(
    "stopwords_removed", col("words_before") - col("words_after")
).withColumn(
    "removal_percentage", (col("stopwords_removed") / col("words_before")) * 100
)

# статистика по удалению
removal_stats = stopword_analysis.groupBy("sentiment").agg(
    avg("words_before").alias("avg_words_before"),
    avg("words_after").alias("avg_words_after"),
    avg("stopwords_removed").alias("avg_stopwords_removed"),
    avg("removal_percentage").alias("avg_removal_percent")
).orderBy("avg_removal_percent", ascending=False)

removal_stats.show()

print("\n   Пример удаления стоп-слов:")
sample_removal = stopword_analysis.select(
    "film_id", "review_num", "sentiment",
    col("words_before").alias("до_удаления"),
    col("words_after").alias("после_удаления"),
    col("stopwords_removed").alias("удалено_слов"),
    col("removal_percentage").alias("процент_удаления"),
    substring("review_text", 1, 60).alias("текст")
).orderBy(col("удалено_слов").desc()).limit(3)

sample_removal.show(truncate=False)

# сохраняем промежуточные результаты
print("\n   Сохранение подготовленных данных...")
df_no_stopwords.write.mode("overwrite").parquet("data/processed/reviews_tokenized.parquet")
print("    Данные сохранены: data/processed/reviews_tokenized.parquet")

   Всего стоп-слов: 169
   Из них общих: 151
   Контекстных (фильмы): 18

   Примеры стоп-слов:
   Общие: ['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со']
   Контекстные: ['фильм', 'кино', 'смотреть', 'режиссер', 'актер', 'актриса', 'роль', 'сюжет', 'концовка', 'кадр', 'эпизод', 'сцена', 'персонаж', 'герой', 'картина', 'лента', 'просмотр', 'кинолента']

  Эффективность удаления стоп-слов:


+---------+------------------+------------------+---------------------+-------------------+
|sentiment|  avg_words_before|   avg_words_after|avg_stopwords_removed|avg_removal_percent|
+---------+------------------+------------------+---------------------+-------------------+
|      neg| 310.2595675869917|200.40648020602504|   109.85308738096664|  35.41035172509948|
|      neu|295.89243871221817|193.00310130944177|    102.8893374027764|  34.85853558458868|
|      pos|299.73392521955964|197.65829531286394|   102.07562990669571|  34.06615881585567|
+---------+------------------+------------------+---------------------+-------------------+


   Пример удаления стоп-слов:


+-------+----------+---------+-----------+--------------+------------+------------------+------------------------------------------------------------+
|film_id|review_num|sentiment|до_удаления|после_удаления|удалено_слов|процент_удаления  |текст                                                       |
+-------+----------+---------+-----------+--------------+------------+------------------+------------------------------------------------------------+
|463149 |30        |neg      |795        |433           |362         |45.534591194968556|В центре сюжета любовный треугольник: Декс любит Рейчел, Рей|
|42664  |7         |neu      |751        |394           |357         |47.53661784287617 |Что же посмотреть перед Новым годом, если не всеми любимую к|
|710276 |40        |neg      |764        |408           |356         |46.596858638743456|Так много сказано, так мало сделано. Наверное, я выросла из |
+-------+----------+---------+-----------+--------------+------------+------------------+-----

    Данные сохранены: data/processed/reviews_tokenized.parquet


#### Дополнительная очистка и нормализация

In [38]:
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType
import re

# функция для дополнительной очистки токенов
def clean_tokens(tokens):
    """Очистка токенов: удаление коротких слов, чисел, пунктуации"""
    cleaned = []
    for token in tokens:
        # удаляем токены короче 2 символов
        if len(token) < 2:
            continue
        
        # удаляем токены, состоящие только из цифр
        if token.isdigit():
            continue
        
        # удаляем токены с цифрами
        if any(char.isdigit() for char in token):
            continue
        
        # приводим к нижнему регистру (на всякий случай)
        token = token.lower()
        
        # удаляем оставшиеся спецсимволы
        token = re.sub(r'[^а-яёa-z]', '', token)
        
        if token and len(token) >= 2:
            cleaned.append(token)
    
    return cleaned

# регистрируем UDF
clean_tokens_udf = udf(clean_tokens, ArrayType(StringType()))

# применяем дополнительную очистку
df_final_tokens = df_no_stopwords.withColumn(
    "words_final",
    clean_tokens_udf(col("words_without_stopwords"))
)

# фильтруем отзывы с слишком маленьким количеством слов
initial_count = df_final_tokens.count()
df_final_tokens = df_final_tokens.filter(size(col("words_final")) >= 3)
final_count = df_final_tokens.count()

print(f"   После фильтрации (минимум 3 слова): {final_count:,} из {initial_count:,} отзывов")
print(f"   Удалено: {initial_count - final_count:,} отзывов")

# анализ финальных токенов
print("\n   Итоговая статистика по токенам:")
final_stats = df_final_tokens.withColumn("final_word_count", size(col("words_final"))) \
                            .groupBy("sentiment") \
                            .agg(
                                avg("final_word_count").alias("avg_final_words"),
                                stddev("final_word_count").alias("std_final_words"),
                                min("final_word_count").alias("min_final_words"),
                                max("final_word_count").alias("max_final_words"),
                                sql_count("*").alias("review_count")
                            ).orderBy("avg_final_words", ascending=False)

final_stats.show()

print("\n   Сохранение финальных подготовленных данных...")
df_final_tokens.select(
    "film_id", "review_num", "sentiment", "review_text",
    "review_length", "word_count", "words_final"
).write.mode("overwrite").parquet("data/processed/reviews_prepared_for_ml.parquet")

print("  Данные сохранены: data/processed/reviews_prepared_for_ml.parquet")

   После фильтрации (минимум 3 слова): 109,998 из 109,998 отзывов
   Удалено: 0 отзывов

   Итоговая статистика по токенам:


+---------+------------------+------------------+---------------+---------------+------------+
|sentiment|   avg_final_words|   std_final_words|min_final_words|max_final_words|review_count|
+---------+------------------+------------------+---------------+---------------+------------+
|      neg|195.83787506737738| 99.34245729667747|              8|            525|       16697|
|      pos| 193.5098168166934|101.26014404459588|              5|            571|       72987|
|      neu|189.14157723737324|102.35256083840292|              6|            538|       20314|
+---------+------------------+------------------+---------------+---------------+------------+


   Сохранение финальных подготовленных данных...


  Данные сохранены: data/processed/reviews_prepared_for_ml.parquet


### TF-IDF векторизация

In [39]:
from pyspark.ml.feature import HashingTF, IDF, CountVectorizer
from pyspark.ml import Pipeline
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType, DoubleType
import numpy as np

print("   Загрузка подготовленных данных...")
df_prepared = spark.read.parquet("data/processed/reviews_prepared_for_ml.parquet")
print(f"   Загружено отзывов: {df_prepared.count():,}")

# HashingTF + IDF (эффективно для больших данных)
print("\n   а) Метод 1: HashingTF + IDF (рекомендуется для больших данных)")

# определяем размер фичевого пространства
num_features = 1024  # 2^10 - хороший баланс

hashing_tf = HashingTF(
    inputCol="words_final",
    outputCol="raw_features",
    numFeatures=num_features
)

idf = IDF(
    inputCol="raw_features",
    outputCol="tfidf_features",
    minDocFreq=2  # слово должно встречаться минимум в 2 документах
)

# создаем пайплайн
pipeline_tfidf = Pipeline(stages=[hashing_tf, idf])

print(f"   Обучение TF-IDF модели на {num_features} фичах...")
tfidf_model = pipeline_tfidf.fit(df_prepared)
df_tfidf = tfidf_model.transform(df_prepared)

print(f"   TF-IDF векторы созданы для {df_tfidf.count():,} документов")

print("\n   б) Анализ TF-IDF векторов:")

# функция для анализа вектора
def analyze_vector(vector):
    if vector is None:
        return (0, 0.0, 0.0, 0.0)
    
    values = vector.toArray()
    non_zero = np.count_nonzero(values)
    avg_value = np.mean(values) if len(values) > 0 else 0.0
    max_value = np.max(values) if len(values) > 0 else 0.0
    min_value = np.min(values) if len(values) > 0 else 0.0
    
    return (int(non_zero), float(avg_value), float(max_value), float(min_value))

# регистрируем UDF
analyze_vector_udf = udf(analyze_vector, 
                        StructType([
                            StructField("non_zero", IntegerType()),
                            StructField("avg_value", DoubleType()),
                            StructField("max_value", DoubleType()),
                            StructField("min_value", DoubleType())
                        ]))

# анализируем векторы
df_vector_analysis = df_tfidf.withColumn("vector_stats", analyze_vector_udf(col("tfidf_features")))

# извлекаем статистику
df_vector_analysis = df_vector_analysis \
    .withColumn("non_zero_features", col("vector_stats.non_zero")) \
    .withColumn("avg_feature_value", col("vector_stats.avg_value")) \
    .withColumn("max_feature_value", col("vector_stats.max_value")) \
    .withColumn("min_feature_value", col("vector_stats.min_value"))

print("\n   Статистика по TF-IDF векторам:")
tfidf_stats = df_vector_analysis.groupBy("sentiment").agg(
    avg("non_zero_features").alias("avg_non_zero"),
    stddev("non_zero_features").alias("std_non_zero"),
    avg("avg_feature_value").alias("avg_feature_value"),
    sql_count("*").alias("review_count")
).orderBy("avg_non_zero", ascending=False)

tfidf_stats.show()

print("\n   в) Пример TF-IDF вектора:")
sample_vector = df_vector_analysis.select(
    "film_id", "review_num", "sentiment",
    "non_zero_features", "avg_feature_value",
    substring("review_text", 1, 50).alias("text_preview")
).limit(1).collect()[0]

print(f"   Фильм {sample_vector['film_id']}, отзыв {sample_vector['review_num']}")
print(f"   Тональность: {sample_vector['sentiment']}")
print(f"   Текст: {sample_vector['text_preview']}...")
print(f"   Ненулевых фич: {sample_vector['non_zero_features']}")
print(f"   Среднее значение фичи: {sample_vector['avg_feature_value']:.4f}")

# сохраняем TF-IDF векторы
print("\n   г) Сохранение TF-IDF результатов...")
df_tfidf.select(
    "film_id", "review_num", "sentiment", "review_text",
    "tfidf_features"
).write.mode("overwrite").parquet("data/processed/reviews_tfidf_vectors.parquet")

print("  TF-IDF векторы сохранены: data/processed/reviews_tfidf_vectors.parquet")

   Загрузка подготовленных данных...
   Загружено отзывов: 109,998

   а) Метод 1: HashingTF + IDF (рекомендуется для больших данных)
   Обучение TF-IDF модели на 1024 фичах...


   TF-IDF векторы созданы для 109,998 документов

   б) Анализ TF-IDF векторов:

   Статистика по TF-IDF векторам:


+---------+------------------+-----------------+------------------+------------+
|sentiment|      avg_non_zero|     std_non_zero| avg_feature_value|review_count|
+---------+------------------+-----------------+------------------+------------+
|      neg|159.99520872012937|70.92134842708138|0.3337238906759899|       16697|
|      pos|157.64652609368792| 72.0375654930883|0.3298302820759359|       72987|
|      neu|154.97297430343605|73.95257612901256|0.3227918246243086|       20314|
+---------+------------------+-----------------+------------------+------------+


   в) Пример TF-IDF вектора:


Traceback (most recent call last):                                              
  File "/home/shoose/anaconda3/lib/python3.13/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 233, in manager
    code = worker(sock, authenticated)
  File "/home/shoose/anaconda3/lib/python3.13/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 87, in worker
    outfile.flush()
    ~~~~~~~~~~~~~^^
BrokenPipeError: [Errno 32] Broken pipe


   Фильм 841263, отзыв 86
   Тональность: pos
   Текст: С момента появления первых фильмов от 'Марвел' про...
   Ненулевых фич: 338
   Среднее значение фичи: 0.7634

   г) Сохранение TF-IDF результатов...


  TF-IDF векторы сохранены: data/processed/reviews_tfidf_vectors.parquet


### Word2Vec для семантических эмбеддингов

In [43]:
from pyspark.ml.feature import Word2Vec
import numpy as np
import builtins
print("   Обучение Word2Vec модели...")

# используем подвыборку для обучения (Word2Vec требует много ресурсов)
sample_size = builtins.min(10000, df_prepared.count())
print(f"   Используем {sample_size:,} отзывов для обучения")

word2vec = Word2Vec(
    vectorSize=100,           # размерность эмбеддинга
    windowSize=3,            # размер окна контекста
    minCount=5,              # минимальная частота слова
    maxIter=10,              # количество итераций
    inputCol="words_final",
    outputCol="word2vec_embeddings",
    seed=42
)

w2v_model = word2vec.fit(df_prepared.limit(sample_size))

print("   Применение Word2Vec ко всем отзывам...")
df_w2v = w2v_model.transform(df_prepared)

print(f"   ✓ Word2Vec эмбеддинги созданы для {df_w2v.count():,} документов")

print("\n   а) Информация о Word2Vec модели:")
print(f"   Размер словаря: {w2v_model.getVectors().count():,} слов")
print(f"   Размерность эмбеддингов: {word2vec.getVectorSize()}")
print(f"   Размер окна контекста: {word2vec.getWindowSize()}")

print("\n   б) Анализ эмбеддингов документов:")

def analyze_embedding(embedding):
    if embedding is None:
        return (0.0, 0.0, 0.0)
    
    values = np.array(embedding)
    return (float(values.mean()), float(values.std()), float(values.max()))

analyze_embedding_udf = udf(analyze_embedding,
                           StructType([
                               StructField("mean", DoubleType()),
                               StructField("std", DoubleType()),
                               StructField("max", DoubleType())
                           ]))

df_embedding_analysis = df_w2v.withColumn("embedding_stats", analyze_embedding_udf(col("word2vec_embeddings")))

df_embedding_analysis = df_embedding_analysis \
    .withColumn("embedding_mean", col("embedding_stats.mean")) \
    .withColumn("embedding_std", col("embedding_stats.std")) \
    .withColumn("embedding_max", col("embedding_stats.max"))

print("\n   Статистика по эмбеддингам документов:")
embedding_stats = df_embedding_analysis.groupBy("sentiment").agg(
    avg("embedding_mean").alias("avg_mean"),
    avg("embedding_std").alias("avg_std"),
    sql_count("*").alias("review_count")
).orderBy("avg_mean", ascending=False)

embedding_stats.show()

print("\n   в) Сохранение Word2Vec результатов...")

w2v_model.write().overwrite().save("models/word2vec_model") 
print("   ✓ Модель сохранена (с перезаписью): models/word2vec_model")

df_w2v.select(
    "film_id", "review_num", "sentiment",
    "word2vec_embeddings"
).write.mode("overwrite").parquet("data/processed/reviews_word2vec_embeddings.parquet")
print("   ✓ Эмбеддинги сохранены: data/processed/reviews_word2vec_embeddings.parquet")

w2v_model.getVectors().write.mode("overwrite").parquet("data/processed/word_vectors.parquet")
print("   ✓ Векторы слов сохранены: data/processed/word_vectors.parquet")

   Обучение Word2Vec модели...
   Используем 10,000 отзывов для обучения


   Применение Word2Vec ко всем отзывам...
   ✓ Word2Vec эмбеддинги созданы для 109,998 документов

   а) Информация о Word2Vec модели:
   Размер словаря: 77,406 слов
   Размерность эмбеддингов: 100
   Размер окна контекста: 3

   б) Анализ эмбеддингов документов:

   Статистика по эмбеддингам документов:


+---------+--------------------+--------------------+------------+
|sentiment|            avg_mean|             avg_std|review_count|
+---------+--------------------+--------------------+------------+
|      pos|-1.86855214510668...|0.043701505723708935|       72987|
|      neu|-3.30615228639647...| 0.04337389870245893|       20314|
|      neg|-5.01391402339131...| 0.04235123181779065|       16697|
+---------+--------------------+--------------------+------------+


   в) Сохранение Word2Vec результатов...


25/12/19 17:23:00 WARN TaskSetManager: Stage 597 contains a task of very large size (3079 KiB). The maximum recommended task size is 1000 KiB.


   ✓ Модель сохранена (с перезаписью): models/word2vec_model


25/12/19 17:23:03 WARN TaskSetManager: Stage 601 contains a task of very large size (5850 KiB). The maximum recommended task size is 1000 KiB.


   ✓ Эмбеддинги сохранены: data/processed/reviews_word2vec_embeddings.parquet
   ✓ Векторы слов сохранены: data/processed/word_vectors.parquet
